In [ ]:
import os
from caret_analyze import Architecture, Application, check_procedure, Lttng
from caret_analyze.plot import message_flow
from caret_analyze.plot import chain_latency

from bokeh.plotting import output_notebook, figure, show
output_notebook()

In [ ]:
tracing_log_path = os.environ['HOME'] + '/.ros/tracing/caret_sample'
architecture_org_path = 'architecture.yaml'
architecture_fixed_path = 'sample_0.yaml'

In [ ]:
lttng = Lttng(tracing_log_path, force_conversion=True)
arch = Architecture('lttng', tracing_log_path)
arch.export(architecture_org_path, force=True)

In [ ]:
arch = Architecture('yaml', architecture_org_path)
paths = arch.search_paths(
    '/node_src',
    '/node_dst',
)

path = paths[0]
path.summary.pprint()

arch.add_path('target_path', path)
arch.export(architecture_org_path, force=True)

In [ ]:
# UNDEFINED -> use_latest_message
arch = Architecture('yaml', architecture_fixed_path)
path = arch.get_path('target_path')
path.verify()

In [ ]:
app = Application(arch, lttng)
path = app.get_path('target_path')
message_flow(path, granularity='node', treat_drop_as_delay=False)

In [ ]:
chain_latency(path, granularity='node', treat_drop_as_delay=False, lstrip_s=0.1, rstrip_s=0.1)

In [ ]:
t, latency_ns = path.to_timeseries(remove_dropped=False, treat_drop_as_delay=False, lstrip_s=0.1, rstrip_s=0.1)
latency_ms = latency_ns * 1.0e-6
p = figure()
p.line(t, latency_ms)
show(p)

In [ ]:
bins, hist = path.to_histogram(treat_drop_as_delay=False, lstrip_s=0.1, rstrip_s=0.1)
p = figure()
p.step(hist[1:], bins)
show(p)